### setup

In [12]:
import pandas as pd

In [13]:
def addWinnerLoser(df):
    #this adds "winner" and "loser" columns to the dataframe.
    winner = lambda row: row.p1 if ("Won" in row.result or "BYE" in row.result) \
        else None if ("L_L" in row.result or "L-L" in row.result) \
        else row.p2 if "Lost" in row.result \
        else "Draw" if ("Draw" in row.result or "Drew" in row.result) \
        else None
    loser = lambda row: row.p1 if "Lost" in row.result \
        else row.p2 if "Won" in row.result \
        else "Draw" if ("Draw" in row.result or "Drew" in row.result) \
        else None
    df["winner"] = df.apply(winner, axis=1)
    df["loser"] = df.apply(loser, axis=1)
    return df

In [14]:
stands = pd.read_csv("stands.csv", index_col=0)
decks = pd.read_csv("decks.csv", index_col=0)
raw_pairs = pd.read_csv("raw-pairs.csv",
                       names=["event", "format", "round", "table", "p1", "result", "p2"])

My scraping scripts naturally will change FIRST LAST into LAST, FIRST for purposes of combining with the Elo database. This is unfortunately not great here because stands and decks come from Melee directly and involve raw names. So I need to undo this. Some names don't reverse so easily, so there are manual fixes.

In [15]:
manual = {
    'Alex Purdy': "Alex Purdy",
    "David Alexander Gurrola": 'Alexander Gurrola, David',
    "Juan Astiazaran Ramos": 'Astiazaran Ramos, Juan',
    "John Madison Ballington": 'Ballington, John Madison',
    "Sky Bauerschmidt Sweeney": 'Bauerschmidt Sweeney, Sky',
    "Joe Briones III": 'Briones III, Joe',
    "Zoe Elisa Clapacs": 'Clapacs, Zoe Elisa',
    "Luis Miguel Collado": 'Collado, Luis Miguel',
    "Ethan De Luna": 'De Luna, Ethan',
    "Juan Del Cid": 'Del Cid, Juan',
    "Damian Del Nero": 'Del Nero, Damian',
    "Crystal  Felts": 'Felts, Crystal ',
    "Mikey  Hopkins": 'Hopkins, Mikey ',
    "Kyro Sewall": 'Kyro Sewall',
    "dahun": 'Leigh, Donald',
    "Ryan Arash Marsh": 'Marsh, Ryan Arash',
    "Gerardo Mendoza Cruz": 'Mendoza Cruz, Gerardo',
    "Michael McKeague-Foster": 'Michael McKeague-Foster',
    "Ben A Miller": 'Miller, Ben A',
    "Hi, I'm Peo": "Peo, Hi, I'm",
    "Javier Perez Gonzalez": 'Perez Gonzalez, Javier',
    "John Puglisi Clark": 'Puglisi Clark, John',
    'Nick "The Honse Man" Roller': 'Roller, Nick "The Honse Man"',
    "Benjamin  Taylor": 'Taylor, Benjamin ',
    "Robert Wagner Krankel": 'Wagner Krankel, Robert',
    "Mason  Wede": 'Wede, Mason ',
    "Pedro da Silva": 'da Silva, Pedro',
}
    
def rev(name):
    if name in manual:
        return manual[name]
    elif len(name.split()) == 2:
        first, last = name.split()[0].strip(), name.split()[1].strip()
        return f"{last}, {first}"
    else:
        return name

In [16]:
stands["pilot_rev"] = stands["pilot"].apply(rev)

### Who played the most Izzet mirrors?

In [17]:
with_players = raw_pairs.merge(
    stands[["pilot_rev", "archetype"]], how="left", left_on="p1", right_on="pilot_rev").merge(
    stands[["pilot_rev", "archetype"]], how="left", left_on="p2", right_on="pilot_rev", suffixes=("_1", "_2")
)
archestands = with_players[["round", "table", "p1", "res", "p2", "archetype_1", "archetype_2"]]
archestands_p1 = archestands.copy().rename(
    {"p1": "player", "p2": "opp", "archetype_1": "archetype", "archetype_2": "opp_archetype"}, axis=1)
archestands_p2 = archestands.copy().rename(
    {"p2": "player", "p1": "opp", "archetype_2": "archetype", "archetype_1": "opp_archetype"}, axis=1)
archestands_stacked = pd.concat([archestands_p1, archestands_p2]).sort_index()
archestands_stacked

KeyError: "['res'] not in index"

In [18]:
decks2 = decks.merge(stands[["pilot_rev", "mp", "place"]], how="left", left_on="pilot", right_on="pilot_rev")
cutter_pilots = decks2[decks2["card"] == "Cori-Steel Cutter"]["pilot"].unique()
izzet_pilots = decks2[(decks2["card"] == "Stormchaser's Talent") & decks2["pilot"].isin(cutter_pilots)]["pilot"].unique()
len(izzet_pilots)

454

In [19]:
ans = archestands_stacked[archestands_stacked["opp"].isin(izzet_pilots)].groupby("player").agg(prowess=("round", "count")).sort_values(by="prowess")
ans[ans["prowess"] >= 9]

NameError: name 'archestands_stacked' is not defined

In [20]:
archestands_stacked[(archestands_stacked["opp"].isin(izzet_pilots)) & (
    archestands_stacked["player"] == "Prost, Andrejs")]

NameError: name 'archestands_stacked' is not defined

### Who did the best with 3-4 copies of a unique card?

In [21]:
all_card_stats = decks2.groupby("card").agg(
    total_copies=("total", "sum"),
    unique_pilots=("pilot", "nunique")
)
uniques = all_card_stats[(all_card_stats["unique_pilots"] == 1) & (all_card_stats["total_copies"] >= 3)]
decks2[(decks2["card"].isin(uniques.index)) & (decks2["mp"] >= 18)][["mp", "place", "pilot", "archetype", "card", "total", "loc"]]

,mp,place,pilot,archetype,card,total,loc
4149,30,145,"Marconi, Nick",Esper Oculus,Spectral Denial,3,main
5072,27,176,"Wan, Tingfeng",Temur Otters,Aegis Turtle,4,side
5372,27,187,"Boucher, David",Mono-Black Reanimator,Shadow-Rite Priest,4,main
6753,24,234,Episkey,Temur Cauldron,"Redshift, Rocketeer Chief",4,main
6898,24,239,"Belfatto, Michael",Orzhov Amalia,Essence Channeler,4,main
6908,24,239,"Belfatto, Michael",Orzhov Amalia,Starscape Cleric,4,main
6953,24,241,Freohr,Quintorius Combo,Etali's Favor,4,main
6954,24,241,Freohr,Quintorius Combo,Geological Appraiser,4,main
6955,24,241,Freohr,Quintorius Combo,Autonomous Assembler,4,main
6958,24,241,Freohr,Quintorius Combo,Quintorius Kand,4,main


### Breakdown of prowess success by number of lands

To get the "is this a land?" data I brought in my copy of the Scryfall database. You'll need to download this yourself from https://scryfall.com/docs/api/bulk-data and adjust the next cell to change the path to where it 

In [22]:
sfall = pd.read_json("../../scryfall-default-cards.json")

FileNotFoundError: File ../../scryfall-default-cards.json does not exist

In [23]:
sfall_trunc = sfall[["name", "type_line", "set"]].dropna().groupby("name").first()
lands = sfall_trunc[sfall_trunc["type_line"].str.contains("Land")]

NameError: name 'sfall' is not defined

In [24]:
decks2["is_land"] = decks2["card"].apply(lambda c: c in lands.index.values)

NameError: name 'lands' is not defined

In [48]:
decks2["land_mask"] = decks2.apply(lambda row: row.total * row.is_land, axis=1)
land_counts = decks2.groupby("pilot_rev", as_index=False).agg(
    archetype=("archetype", "first"),
    lands=("land_mask", "sum")
)
izzet_with_lands = land_counts[land_counts["pilot_rev"].isin(izzet_pilots)]
stock_ups = decks2[decks2["card"] == "Stock Up"].groupby("pilot_rev").agg(stock_ups=("total", "sum"))
izzet_with_lands = izzet_with_lands.merge(stock_ups, how="left", on="pilot_rev").fillna(0)

In [52]:
pairs_with_wl = addWinnerLoser(raw_pairs)
wins = pairs_with_wl["winner"].value_counts()
losses = pairs_with_wl["loser"].value_counts()
izzet_with_lands_recs = izzet_with_lands.merge(
    wins, how="left", left_on="pilot_rev", right_index=True).merge(
    losses, how="left", left_on="pilot_rev", right_index=True
).fillna(0)
ans = izzet_with_lands_recs.groupby("lands").agg(
    pilots=("pilot_rev", "count"),
    wins=("winner", "sum"),
    losses=("loser", "sum"),
    avg_stock_ups=("stock_ups", "mean")
)
ans["w_pct"] = ans["wins"] / (ans["wins"] + ans["losses"])
ans["wins"] = ans["wins"].astype(int)
ans[["pilots", "wins", "losses", "w_pct", "avg_stock_ups"]]

,pilots,wins,losses,w_pct,avg_stock_ups
lands,,,,,
19,7,26,32,0.448276,2.000000
20,175,741,743,0.499326,3.542857
21,269,1247,1140,0.522413,3.858736
22,3,12,12,0.500000,3.333333


### Drake Hatcher copies/locations

In [76]:
hatchers = decks2[decks2["card"] == "Drake Hatcher"].copy()
agg_hatchers = hatchers.groupby("pilot_rev").agg(
    hatchers=("total", "sum"),
    counts=("total", lambda s: tuple(s.values)),
    locs=("loc", lambda s: tuple(s.values))
)

def locs(row):
    if row.locs[0] == "main":
        if len(row.locs) == 1: return row.counts + (0,)
        else: return row.counts
    else:
        return (0,) + row.counts

agg_hatchers["locations"] = agg_hatchers.apply(locs, axis=1)
agg_hatchers["mains"] = agg_hatchers["locations"].apply(lambda t: t[0])
agg_hatchers["sides"] = agg_hatchers["locations"].apply(lambda t: t[1])

In [99]:
izzet_hatchers = izzet_with_lands_recs.merge(agg_hatchers, how="left", left_on="pilot_rev", right_index=True)
izzet_hatchers["hatchers"] = izzet_hatchers["hatchers"].fillna(0).astype(int)
izzet_hatchers["mains"] = izzet_hatchers["mains"].fillna(0).astype(int)
izzet_hatchers["sides"] = izzet_hatchers["sides"].fillna(0).astype(int)
izzet_hatchers["locations"] = izzet_hatchers["locations"].apply(lambda v: v if isinstance(v, tuple) else (0,0))
breakdown = izzet_hatchers.groupby("locations").agg(
    pilots=("pilot_rev", "count"),
    wins=("winner", "sum"),
    losses=("loser", "sum")
)
breakdown["w_pct"] = breakdown["wins"]/(breakdown["wins"]+breakdown["losses"])
breakdown["wins"] = breakdown["wins"].astype(int)
breakdown.sort_index(ascending=False)

,pilots,wins,losses,w_pct
locations,,,,
"(4, 0)",6,45,29,0.608108
"(3, 1)",2,9,9,0.500000
"(3, 0)",20,107,87,0.551546
"(2, 2)",30,188,133,0.585670
"(2, 1)",112,557,476,0.539206
"(2, 0)",51,164,212,0.436170
"(1, 3)",1,3,4,0.428571
"(1, 2)",14,67,59,0.531746
"(1, 1)",13,72,57,0.558140


In [100]:
breakdownb = izzet_hatchers.groupby("mains").agg(
    pilots=("pilot_rev", "count"),
    wins=("winner", "sum"),
    losses=("loser", "sum")
)
breakdownb["w_pct"] = breakdownb["wins"]/(breakdownb["wins"]+breakdownb["losses"])
breakdownb["wins"] = breakdownb["wins"].astype(int)
breakdownb.sort_index(ascending=False)

,pilots,wins,losses,w_pct
mains,,,,
4,6,45,29,0.608108
3,22,116,96,0.547170
2,193,909,821,0.525434
1,39,169,162,0.510574
0,194,787,819,0.490037


In [101]:
breakdownc = izzet_hatchers.groupby("hatchers").agg(
    pilots=("pilot_rev", "count"),
    wins=("winner", "sum"),
    losses=("loser", "sum")
)
breakdownc["w_pct"] = breakdownc["wins"]/(breakdownc["wins"]+breakdownc["losses"])
breakdownc["wins"] = breakdownc["wins"].astype(int)
breakdownc.sort_index(ascending=False)

,pilots,wins,losses,w_pct
hatchers,,,,
4,40,248,179,0.580796
3,154,752,656,0.534091
2,143,644,613,0.512331
1,16,49,60,0.449541
0,101,333,419,0.442819


In [103]:
breakdownd = breakdownb.copy()
breakdownd.loc["yes"] = breakdownb.loc[[1,2,3,4]].sum().T
breakdownd.loc["no"] = breakdownb.loc[0].T
breakdownd = breakdownd.loc[["yes", "no"]].astype(int)
breakdownd["w_pct"] = breakdownd["wins"]/(breakdownd["wins"]+breakdownd["losses"])
breakdownd["wins"] = breakdownd["wins"].astype(int)

breakdownd

,pilots,wins,losses,w_pct
mains,,,,
yes,260,1239,1108,0.527908
no,194,787,819,0.490037
